In [163]:
import importlib

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [164]:
# Manually reloading python module such that
# jupyter reflects changes without kernel restart

import apollo.api.yahoo_api_connector as yac
import apollo.calculations.average_true_range as atr
import apollo.calculations.keltner_channel as kc
import apollo.calculations.mcnicholl_moving_average as mma

importlib.reload(yac)
importlib.reload(mma)
importlib.reload(kc)
importlib.reload(atr);

In [165]:
ticker = "SPY"
start_date = "2024-01-01"
end_date = "2024-03-01"

api_connector = yac.YahooApiConnector(ticker, start_date, end_date)
dataframe = api_connector.request_or_read_prices()

dataframe;

In [166]:
WINDOW_SIZE = 5

mnme_calculator = mma.McNichollMovingAverageCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)

mnme_calculator.calculate_mcnicholl_moving_average()

atr_calculator = atr.AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)

atr_calculator.calculate_average_true_range()

kc_calculator = kc.KeltnerChannelCalculator(
    dataframe=dataframe,
    window_size=5,
    volatility_multiplier=0.5,
)

kc_calculator.calculate_keltner_channel()

dataframe.dropna(inplace=True)

dataframe

,ticker,open,high,low,close,adj close,volume,mnma,tr,atr,lkc_bound,ukc_bound
date,,,,,,,,,,,,
2024-01-12,SPY,477.839996,478.600006,475.230011,476.679993,475.203522,57944000,474.307385,3.369995,5.233992,471.690389,476.924381
2024-01-16,SPY,475.260010,476.609985,473.059998,474.929993,473.458923,85014900,474.273148,3.619995,4.911193,471.817552,476.728744
2024-01-17,SPY,471.820007,472.790009,469.869995,472.290009,470.827118,68843900,473.271218,5.059998,4.940954,470.800741,475.741695
2024-01-18,SPY,474.010010,477.059998,472.420013,476.489990,475.014099,91856200,474.039532,4.769989,4.906761,471.586152,476.492912
2024-01-19,SPY,477.649994,482.720001,476.540009,482.429993,480.935699,110733300,476.702012,6.230011,5.171411,474.116306,479.287717
2024-01-22,SPY,484.010010,485.220001,482.779999,483.450012,481.952576,75844900,478.941422,2.790009,4.695130,476.593856,481.288987
2024-01-23,SPY,484.010010,485.109985,482.890015,484.859985,483.358185,49945300,481.001109,2.219971,4.200098,478.901060,483.101159
2024-01-24,SPY,487.809998,488.769989,484.880005,485.390015,483.886566,81765000,482.603810,3.910004,4.142079,480.532770,484.674850
2024-01-25,SPY,487.579987,488.309998,485.390015,488.029999,486.518372,72525000,484.628298,2.919983,3.897660,482.679468,486.577128


In [167]:
fig = make_subplots(rows=1, cols=1, specs=[[{}]], vertical_spacing=0)

# Plot closing prices
fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["adj close"],
        line={"color": "blue", "width": 1},
        name="Adj close",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["lkc_bound"],
        line={"color": "red", "width": 1},
        name="MNMA",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe["ukc_bound"],
        line={"color": "red", "width": 1},
        name="Upper Bound",
    ),
    row=1,
    col=1,
)

fig.update_layout(
    title={"text": f"{ticker}", "x": 0.5},
    height=650,
    plot_bgcolor="#EFF5F8",
)

fig.update_yaxes(visible=False, secondary_y=True)